In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')

In [2]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

In [3]:
import json

from autogen_ext.models import AzureOpenAIChatCompletionClient

# Create the client with type-checked environment variables
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True
    }
)

/var/folders/v1/cv54pg7n7rdfjb_37_f20cd00000gn/T/ipykernel_22744/731393293.py:6: DeprecationWarning: autogen_ext.models.AzureOpenAIChatCompletionClient moved to autogen_ext.models.openai.AzureOpenAIChatCompletionClient. This alias will be removed in 0.4.0.
  client = AzureOpenAIChatCompletionClient(
/Users/sjuratovic/repos/autogen-experiments/.venv/lib/python3.10/site-packages/autogen_ext/models/openai/_openai_client.py:1061: UserWarning: Previous behavior of using the model name as the deployment name is deprecated and will be removed in 0.4. Please specify azure_deployment
  client = _azure_openai_client_from_config(copied_args)


In [4]:
from autogen_core.components.models import UserMessage

# Define the user message
messages = [
    UserMessage(content="What is 16 + 32?", source="user"),
]

# Call the create method on the client, passing the messages and additional arguments
# The extra_create_args dictionary includes the response format as MathReasoning model we defined above
# Providing the response format and pydantic model will use the new parse method from beta SDK
response = await client.create(messages=messages, extra_create_args={"response_format": MathReasoning})

/var/folders/v1/cv54pg7n7rdfjb_37_f20cd00000gn/T/ipykernel_22744/3799510949.py:5: DeprecationWarning: autogen_core.components.models.UserMessage moved to autogen_core.models.UserMessage. This alias will be removed in 0.4.0.
  UserMessage(content="What is 16 + 32?", source="user"),


In [5]:
from typing import Optional

# Ensure the response content is a valid JSON string before loading it
response_content: Optional[str] = response.content if isinstance(response.content, str) else None
if response_content is None:
    raise ValueError("Response content is not a valid JSON string")

# Print the response content after loading it as JSON
print(json.loads(response_content))

# Validate the response content with the MathReasoning model
MathReasoning.model_validate(json.loads(response_content))

{'steps': [{'explanation': 'Start with the number 16.', 'output': '16'}, {'explanation': 'Add the second number, which is 32, to the first number 16.', 'output': '16 + 32 = 48'}], 'final_answer': '48'}


MathReasoning(steps=[Step(explanation='Start with the number 16.', output='16'), Step(explanation='Add the second number, which is 32, to the first number 16.', output='16 + 32 = 48')], final_answer='48')